In [1]:
from probabilityPlus import *
from prettytable import *
from utils import print_table
import itertools

In [89]:
#em que  𝑅  é uma Rede de Bayes,  𝑋  e  𝑌  são duas variáveis diferentes na rede  𝑅 , e  𝐸  é um conjunto, 
#possivelmente vazio, de variáveis da Rede de Bayes  𝑅  que não contem nem  𝑋  nem  𝑌 . 
#Essa função verifica se  𝑋  é condicionalmente independente de  𝑌  dado o conjunto de variáveis  𝐸 , 
#devolvendo um valor booleano.
def cond_indep(X, Y, E, R):
    l = []
    path(l, X, Y, R)
    for p in l:
        if state(p, E, R):
            return False
    return True

def state(p, E, R):
    l = p[:3]
    t = p[1:]
    if(len(l) < 3):
        return True
    if inactive(l, E, R): #True if inactive, False otherwise
        return False
    elif len(t) >= 3:
        state(t, E, R)
    else:
        return True
    
def inactive(l, E, R):
    if efeitoCausal(l, E, R):
        if l[1] in E:
            return True
        else:
            return False
    elif efeitoComum(l, E, R):
        if l[1] in E:
            return False
        for node in R.variable_node(l[1]).children:
            if node in E: # check if any descendants of middle cenas is in evidence
                return False
    else:
        if l[1] in E:
            return True

def efeitoCausal(l, E, R): #True if it is
    return (l[0] in R.variable_node(l[1]).parents and l[1] in R.variable_node(l[2]).parents) or (l[2] in R.variable_node(l[1]).parents and l[1] in R.variable_node(l[0]).parents)

def efeitoComum(l, E, R): #True if it is
    return l[0] in R.variable_node(l[1]).parents and l[2] in R.variable_node(l[1]).parents
           
def path(lista, X, Y, R, t = []):
    if X == Y:
        t.append(X)
        lista.append(t)
        return
    
    if X in t:
        return 
    
    children = R.variable_node(X).children 
    if len(children) > 0:
        for child in children:
            l = []
            for x in t:
                l.append(x)
            l.append(X)
            path(lista, child, Y , R, l)
    parents = R.variable_node(X).parents
    if len(parents) > 1:
        for parent in parents:
            if parent != t[-1]:
                l = []
                for x in t:
                    l.append(x)
                l.append(X)
                path(lista, parent, Y, R, l)
    
T, F = True, False

burglary = BayesNet([
    ('A', '', 0.001),
    ('B', 'A', {T: 0.90, F: 0.05}),
    ('D', 'A', {T: 0.90, F: 0.05}),
    ('E', 'B D', {(T, T): 0.95, (T, F): 0.94, (F, T): 0.29, (F, F): 0.001}),
    ('C', 'B', {T: 0.70, F: 0.01})
])

#cond_indep('A', 'C', 'E', burglary)
print(cond_indep('A', 'A', ['A'], burglary))
print(cond_indep('A', 'A', [], burglary))



False
False


In [87]:
burglary = BayesNet([
    ('A', '', 0.001),
    ('B', 'A', {T: 0.90, F: 0.05}),
    ('D', 'A', {T: 0.90, F: 0.05}),
    ('E', 'B D', {(T, T): 0.95, (T, F): 0.94, (F, T): 0.29, (F, F): 0.001}),
    ('C', 'B', {T: 0.70, F: 0.01})
])

#que recebe como argumento uma instância  𝑅  de uma Rede de Bayes 
#e que devolve uma lista com todos os triplos distintos  (𝑋,𝑌,𝐸)  tais que se verifica que  
#𝑋  é condicionalmente independente de  𝑌  dado  𝐸 a rede n  𝑅 .
def todos_cond_indeps(R):
    nodes = []
    ret = []
    for node in R.nodes:
        nodes.append(node.variable)
    E = evi(nodes)
    for nodeX in nodes:
        for nodeY in nodes:
            for ev in E:
                if cond_indep(nodeX, nodeY, ev, R):
                    ret.append((nodeX, nodeY, ev))        
def evi(nodes):
    ret = []
    for L in range(0, len(nodes)+1):
        for subset in itertools.combinations(nodes, L):
            ret.append(list(subset))
    return ret

def isParent(nodeX, nodeY, bayesNet): 
    return nodeY in bayesNet.variable_node(nodeX).parents

#todos_cond_indeps(burglary)
cond_indep('A', 'A', [], burglary)

True